Threestudio-dreamcraft3D

In [ ]:
%cd /notebooks/threestudio
prompt="a anime girl"
slug="a_anime_girl"
name="anya_front"
b_image_path=f"/notebooks/threestudio/load/images/{name}.png"
image_path=f"/notebooks/threestudio/load/images/{name}_rgba.png"

GEODREAM


In [ ]:
%cd /notebooks/threestudio/custom/threestudio-geodream/mv-diffusion/One-2-3-45-by-view
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/stabilityai/stable-zero123/resolve/main/stable_zero123.ckpt?download=true  -o "stable_zero123.ckpt"

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/One-2-3-45/code/resolve/main/sam_vit_h_4b8939.pth -o "sam_vit_h_4b8939.pth"


In [ ]:
%cd /notebooks/threestudio/
!mkdir one2345_weight
%cd one2345_weight
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/One-2-3-45/code/resolve/main/SparseNeuS_demo_v1/exp/lod0/checkpoints/ckpt_215000.pth  -o "ckpt_215000.pth"


In [ ]:
!rm /notebooks/threestudio/custom/threestudio-geodream/mv-diffusion/volume/ -rf

In [ ]:
%cd /notebooks/threestudio/custom/threestudio-geodream/mv-diffusion/One-2-3-45-by-view
#!source /root/miniconda3/bin/activate treestudio
!/root/miniconda3/bin/conda run -n treestudio python run.py \
--model_type "sd-zero123" \
--img_path "$b_image_path" \
--text "$prompt" \
--half_precision

In [ ]:
%cd /notebooks/threestudio
!/root/miniconda3/bin/conda run -n treestudio python launch.py  --config custom/threestudio-geodream/configs/geodream-neus.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt" system.geometry.init_volume_path="/notebooks/threestudio/custom/threestudio-geodream/mv-diffusion/volume/a_anime_girl/con_volume_lod_150.pth" tag="Phase1"


In [ ]:
# --------- Stage 2 (DMTet Geometry Refinement) --------- #
geometry_convert_from='/notebooks/threestudio/outputs/geodream/Phase1@20240303-044843/ckpts/last.ckpt'
# refine geometry
%cd /notebooks/threestudio
!/root/miniconda3/bin/conda run -n treestudio python launch.py --config custom/threestudio-geodream/configs/geodream-dmtet-geometry.yaml --train system.geometry_convert_from={geometry_convert_from} --gpu 0 system.prompt_processor.prompt="$prompt" system.renderer.context_type=cuda system.geometry_convert_override.isosurface_threshold=0.0


Gaussian Splatting + Zero-123

In [ ]:
!/root/miniconda3/envs/treestudio/bin/python launch.py --config custom/threestudio-3dgs/configs/gaussian_splatting_zero123.yaml --train --gpu 0 data.image_path={image_path}


In [ ]:
config="outputs/gs-zero123-sai/256_ultra2_rgba.png@20240211-005213/configs/parsed.yaml"                                                                                                                                                                                                                                                                   
checkpoint="outputs/gs-zero123-sai/256_ultra2_rgba.png@20240211-005213/ckpts/last.ckpt"
!/root/miniconda3/envs/treestudio/bin/python launch.py --config {config} --export --gpu 0 system.prompt_processor.prompt="$prompt" resume={checkpoint}


 DreamCraft3D 

In [ ]:
# preprocess images
%cd custom/threestudio-dreamcraft3D
%run image_preprocess.py {b_image_path} --size 512 --border_ratio 0.0
# if you need image caption
# python image_preprocess.py "examples/hamburger.png" --size 512 --border_ratio 0.0 --need_caption
# if you remove backgounrd using other tools like ClipDrop, you need put the processed image to "examples/hamburger_rgba.png" and add option use_existing_background
# python image_preprocess.py "examples/hamburger.png" --size 512 --border_ratio 0.0 --use_existing_background
%cd  ../..

In [ ]:

# --------- Stage 1 (NeRF & NeuS) --------- #
%run launch.py --config /notebooks/threestudio/custom/threestudio-dreamcraft3D/configs/dreamcraft3d-coarse-nerf-fast.yaml --train system.prompt_processor.prompt="$prompt" data.image_path="$image_path" use_timestamp=False tag="Phase1" name="$name"


In [ ]:
ckpt = f"/notebooks/threestudio/outputs/{name}/Phase1/ckpts/last.ckpt"
%run launch.py --config /notebooks/threestudio/custom/threestudio-dreamcraft3D/configs/dreamcraft3d-coarse-neus-fast.yaml --train system.prompt_processor.prompt="$prompt" data.image_path="$image_path" system.weights="$ckpt" use_timestamp=False tag="Phase2" name="$name"

In [ ]:
# --------- Stage 2 (Geometry Refinement) --------- #
ckpt = f"/notebooks/threestudio/outputs/{name}/Phase2/ckpts/last.ckpt"
!/root/miniconda3/envs/treestudio/bin/python  /notebooks/threestudio/launch.py --config /notebooks/threestudio/custom/threestudio-dreamcraft3D/configs/dreamcraft3d-geometry-fast.yaml --train system.prompt_processor.prompt="$prompt" data.image_path="$image_path" system.geometry_convert_from="$ckpt"  use_timestamp=False tag="Phase3" name="$name"


In [ ]:

# --------- Stage 3 (Texture Refinement) --------- #
ckpt = f"/notebooks/threestudio/outputs/{name}/Phase3/ckpts/last.ckpt"
!/root/miniconda3/envs/treestudio/bin/python  /notebooks/threestudio/launch.py  --config /notebooks/threestudio/custom/threestudio-dreamcraft3D/configs/dreamcraft3d-texture-fast.yaml --train system.prompt_processor.prompt="$prompt" data.image_path="$image_path" system.geometry_convert_from="$ckpt" use_timestamp=False tag="Phase4" name="$name"


In [ ]:

# --------- EXPORT --------- 
config = f"/notebooks/threestudio/outputs/{name}/Phase4/configs/parsed.yaml"
ckpt = f"/notebooks/threestudio/outputs/{name}/Phase4/ckpts/last.ckpt"
cmd = f'''--config "{config}" --export system.prompt_processor.prompt="{prompt}" data.image_path="{image_path}" resume="{ckpt}" system.exporter.context_type=cuda use_timestamp=False tag="Phase5" name="{name}"'''
!/root/miniconda3/envs/treestudio/bin/python  /notebooks/threestudio/launch.py {cmd}


In [ ]:
path = f"/notebooks/threestudio/outputs/{name}/Phase5/save"
%cd {path}
!zip {name}.zip -r it3000-export
%cd /notebooks/threestudio/


OTHER

In [ ]:
%run launch.py --config custom/threestudio-mvimg-gen/configs/stable-zero123.yaml --train --gpu 0 data.image_path=./load/images/catstatue_rgba.png


In [ ]:
name='dragon'

In [ ]:
%cd /notebooks/threestudio
%run launch.py use_timestamp=False tag=Phase1 name={name} --config configs/stable-zero123.yaml --train --gpu 0 data.image_path=./load/images/wyvern2_rgba.png data.batch_size=32 


In [ ]:
%cd /notebooks/threestudio
%run launch.py  --config '/notebooks/threestudio/outputs/zero123-sai/[64, 128, 256]_wyvern2_rgba.png@20231227-065541/configs/parsed.yaml' resume='/notebooks/threestudio/outputs/zero123-sai/[64, 128, 256]_wyvern2_rgba.png@20231227-065541/ckpts/last.ckpt' --export --gpu 0 system.exporter_type=mesh-exporter system.exporter.fmt=obj  system.exporter.context_type=cuda system.geometry.isosurface_method=mc-cpu system.geometry.isosurface_resolution=350 system.geometry.isosurface_threshold=25.


In [ ]:
 #/root/miniconda3/bin/conda run -n dreamgaussian python launch.py --config '/notebooks/threestudio/outputs/mvimg-gen-zero123-sai/catstatue_rgba.png@20231227-020100/configs/parsed.yaml' --export --gpu 0

In [ ]:
!rm -rf ./outputs

In [ ]:
%pip install pydantic fastapi lightning --upgrade

In [ ]:
%run gradio_app.py launch


In [ ]:
import os
%env CUDA_VISIBLE_DEVICES=0
# Set environment variable
name = "dog_robot_side"
prompt = "an 3D render of a cartoon skeleton of a dog robot, detailed, 8k"
last="last"
%cd /notebooks/threestudio

In [ ]:
%run launch.py --config configs/magic123-coarse-sd.yaml --train --gpu 0 data.image_path=load/images/hamburger_rgba.png system.prompt_processor.prompt="a delicious hamburger"


In [ ]:
""" data.default_elevation_deg=0
data.default_azimuth_deg=90
data.default_camera_distance=1
system.freq.guidance_eval=0 """
cmd = f''' --train --gpu 0
--config configs/zero123_64.yaml
data.image_path="load/images/{name}_rgba.png"
name="{name}"
tag=Phase1
system.loggers.wandb.enable=false
use_timestamp=False
data.default_azimuth_deg=0
data.random_camera.batch_size=14
system.loss.lambda_depth=0.05
system.loss.lambda_depth_rel="[0, 0, 0.05, 100]"
system.loss.lambda_normal="[0, 0, 0.05, 100]"
system.geometry.density_blob_scale=8.
system.freq.guidance_eval=37
system.guidance.cond_elevation_deg=0
'''

%run launch.py {cmd}

In [ ]:
# Phase 1.5 - 512 refine
#system.loggers.wandb.project="zero123" system.loggers.wandb.name=${NAME}_Phase1p5

cmd = f'''--config configs/zero123-geometry.yaml --train --gpu 0 
data.image_path=./load/images/{name}_rgba.png 
system.geometry_convert_from=./outputs/{name}/Phase1/ckpts/{last}.ckpt 
use_timestamp=False name={name} tag=Phase1p5'''

%run launch.py {cmd}

In [ ]:
# Phase 2 - dreamfusion
#data.random_camera.batch_size=2 trainer.accumulate_grad_batches=4 system.freq.guidance_eval=0
# system.freq.guidance_eval=0 system.loggers.wandb.enable=false system.loggers.wandb.project="zero123" system.loggers.wandb.name=${NAME}_Phase2
tag="Phase1p5"
last="last"
cmd = f'''--config configs/experimental/imagecondition_zero123nerf.yaml 
--train --gpu 0 data.image_path=./load/images/{name}_rgba.png
system.prompt_processor.prompt="{prompt}" 
system.weights="./outputs/{name}/{tag}/ckpts/{last}.ckpt" 
system.freq.guidance_eval=23
name={name} tag=Phase2 system.loggers.wandb.enable=false  use_timestamp=False'''

%run launch.py {cmd}

In [ ]:

# Phase 2 - dreamfusion
cmd = f'''--config configs/experimental/my_imagecondition_zero123nerf_refine.yaml 
--train --gpu 0  system.prompt_processor.prompt="{prompt}" 
system.geometry_convert_from="./outputs/{name}/{phase}/ckpts/{last}.ckpt" 
name={name} 
tag=Phase2_refine system.loggers.wandb.enable=false 
use_timestamp=False
data.image_path=./load/images/{name}_rgba.png
'''

%run launch.py {cmd}

In [ ]:
tag="Phase1p5"
last="last"
cmd=f'''--config "outputs/{name}/{tag}/configs/parsed.yaml" 
--export --gpu 0 resume="outputs/{name}/{tag}/ckpts/{last}.ckpt" 
system.exporter.context_type=cuda  system.exporter_type=mesh-exporter'''
%run launch.py {cmd}

TEXT MESH

In [ ]:
# uses DeepFloyd IF, requires ~15GB VRAM
cmd= f'''
--config configs/textmesh-if.yaml 
--train -
-gpu 0 tag=DRAGON system.prompt_processor.prompt="{prompt}"
system.prompt_processor_type="stable-diffusion-prompt-processor"
system.guidance_type="stable-diffusion-guidance"
system.guidance.pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"
system.prompt_processor.pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"
'''
%run launch.py {cmd}

In [ ]:
tag="DRAGON@20230726-085910"
name="textmesh-if"
cmd=f'--config "outputs/{name}/{tag}/configs/parsed.yaml" --export --gpu 0 resume="outputs/{name}/{tag}/ckpts/last.ckpt" system.exporter.context_type=cuda  system.exporter_type=mesh-exporter'
%run launch.py {cmd}

CONTROL 4D

In [ ]:
cmd = f'''--config configs/control4d-static.yaml --train --gpu 0 
data.dataroot="YOUR_DATAROOT/twindom" 
system.prompt_processor.prompt="{prompt}" 
name={name} 
tag=Phase1 
system.loggers.wandb.enable=false 
use_timestamp=False'''

%run launch.py {cmd}

MAGIC123

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
workspace ="dog_robot_side"
image_path = f'/notebooks/threestudio/load/images/{workspace}_rgba.png'
image_path_without_rgba = image_path.replace('_rgba', '')
prompt = '''
high quality amazing detailed art of 
an 3D stylized dog robot
'''
# If the image contains non-front-facing objects, specifying the approximate elevation 
# and azimuth angle by setting data.default_elevation_deg and data.default_azimuth_deg can be helpful. 
# In threestudio, top is elevation +90 and bottom is elevation -90; left is azimuth -90 and right is azimuth +90.
default_elevation_deg = 0
default_azimuth_deg = -85
%cd /notebooks/threestudio

In [ ]:
%pip install opencv-python==4.8.0.76 numpy==1.23 --force-reinstall --upgrade

In [ ]:
%run launch.py --config configs/magic123-coarse-sd.yaml --train --gpu 0 \
data.image_path="{image_path}" system.prompt_processor.prompt="{prompt}" \
tag=Phase1 data.default_azimuth_deg={default_azimuth_deg} data.default_elevation_deg={default_elevation_deg} \
data.requires_depth=True data.requires_normal=True \
use_timestamp=False \
system.loss.lambda_depth=0.05 system.loss.lambda_depth_rel="[0, 0, 0.05, 100]" system.loss.lambda_normal="[0, 0, 0.05, 100]"


In [ ]:
# Zero123 + Stable Diffusion, ~10GB VRAM
# data.image_path must point to a 4-channel RGBA image
# system.prompt_proessor.prompt must be specified
%run  launch.py --config configs/magic123-refine-sd.yaml --train --gpu 0 data.image_path="{image_path}" \
use_timestamp=False \
system.prompt_processor.prompt="{prompt}" system.geometry_convert_from="outputs/{workspace}/Phase1/ckpts/last.ckpt" tag=Phase2 


In [ ]:
# if you're unsatisfied with the surface extracted using the default threshold (25)
# you can specify a threshold value using `system.geometry_convert_override`
# decrease the value if the extracted surface is incomplete, increase if it is extruded
%run launch.py --config configs/magic123-refine-sd.yaml --train --gpu 0 data.image_path={image_path} \
use_timestamp=False \
system.prompt_processor.prompt={prompt} system.geometry_convert_from="outputs/{workspace}/Phase2/ckpts/last.ckpt"  system.geometry_convert_override.isosurface_threshold=10.